# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("cal_housing_clean.csv")

In [3]:
data.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
data.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [5]:
x_data = data.drop("medianHouseValue", axis=1)
labels = data["medianHouseValue"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x_data, labels, test_size=0.3)

In [7]:
X_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
7991,40.0,1301.0,342.0,954.0,336.0,2.3804
11027,31.0,2016.0,409.0,1095.0,405.0,3.8681
10343,16.0,2022.0,254.0,789.0,270.0,8.4112
18858,31.0,602.0,153.0,112.0,47.0,1.0667
13660,17.0,1779.0,292.0,1006.0,293.0,4.6708


In [8]:
X_train.index

Int64Index([ 7991, 11027, 10343, 18858, 13660,  5745,  4089, 17995, 19661,
             5754,
            ...
             1040,  6798,  6774, 11760, 16864,  4612,  9060, 18937,  3010,
             7906],
           dtype='int64', length=14448)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()

In [11]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
X_train = pd.DataFrame(data=scaler.transform(X_train), columns=X_train.columns, index=X_train.index)

In [13]:
X_test = pd.DataFrame(data=scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [14]:
data.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [15]:
import tensorflow as tf
print(tf.__version__)

1.9.0


In [16]:
median_age = tf.feature_column.numeric_column("housingMedianAge")
rooms = tf.feature_column.numeric_column("totalRooms")
bedrooms = tf.feature_column.numeric_column("totalBedrooms")
population = tf.feature_column.numeric_column("population")
households = tf.feature_column.numeric_column("households")
median_income = tf.feature_column.numeric_column("medianIncome")

In [17]:
feat_cols = [median_age, rooms, bedrooms, population, households, median_income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [18]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=16, 
                                                 num_epochs=1000, shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [19]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6,6], feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Juancho\\AppData\\Local\\Temp\\tmplfhg0puc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000140A4C04240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [24]:
model.train(input_fn=input_func, steps=100000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Juancho\AppData\Local\Temp\tmplfhg0puc\model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 100000 into C:\Users\Juancho\AppData\Local\Temp\tmplfhg0puc\model.ckpt.
INFO:tensorflow:loss = 167347750000.0, step = 100000
INFO:tensorflow:global_step/sec: 273.504
INFO:tensorflow:loss = 115166870000.0, step = 100100 (0.367 sec)
INFO:tensorflow:global_step/sec: 459.185
INFO:tensorflow:loss = 40730034000.0, step = 100200 (0.217 sec)
INFO:tensorflow:global_step/sec: 532.46
INFO:tensorflow:loss = 98890200000.0, step = 100300 (0.189 sec)
INFO:tensorflow:global_step/sec: 544.035
INFO:tensorflow:loss = 143108360000.0, step = 100400 (0.183 sec)
INFO:tensorflow:global_step/sec: 552.825
INFO:tensorflow:loss = 887707

INFO:tensorflow:global_step/sec: 588.836
INFO:tensorflow:loss = 187627570000.0, step = 107400 (0.171 sec)
INFO:tensorflow:global_step/sec: 588.838
INFO:tensorflow:loss = 62583770000.0, step = 107500 (0.169 sec)
INFO:tensorflow:global_step/sec: 578.628
INFO:tensorflow:loss = 101585670000.0, step = 107600 (0.173 sec)
INFO:tensorflow:global_step/sec: 575.301
INFO:tensorflow:loss = 55618100000.0, step = 107700 (0.174 sec)
INFO:tensorflow:global_step/sec: 513.347
INFO:tensorflow:loss = 132662166000.0, step = 107800 (0.195 sec)
INFO:tensorflow:global_step/sec: 572.012
INFO:tensorflow:loss = 133794280000.0, step = 107900 (0.175 sec)
INFO:tensorflow:global_step/sec: 592.323
INFO:tensorflow:loss = 119003010000.0, step = 108000 (0.169 sec)
INFO:tensorflow:global_step/sec: 592.322
INFO:tensorflow:loss = 135164290000.0, step = 108100 (0.169 sec)
INFO:tensorflow:global_step/sec: 599.412
INFO:tensorflow:loss = 102209280000.0, step = 108200 (0.167 sec)
INFO:tensorflow:global_step/sec: 581.993
INFO:te

INFO:tensorflow:global_step/sec: 595.849
INFO:tensorflow:loss = 131518650000.0, step = 115200 (0.169 sec)
INFO:tensorflow:global_step/sec: 588.837
INFO:tensorflow:loss = 151533810000.0, step = 115300 (0.169 sec)
INFO:tensorflow:global_step/sec: 595.848
INFO:tensorflow:loss = 98761000000.0, step = 115400 (0.168 sec)
INFO:tensorflow:global_step/sec: 588.838
INFO:tensorflow:loss = 32391390000.0, step = 115500 (0.170 sec)
INFO:tensorflow:global_step/sec: 592.321
INFO:tensorflow:loss = 115392890000.0, step = 115600 (0.169 sec)
INFO:tensorflow:global_step/sec: 599.417
INFO:tensorflow:loss = 62639063000.0, step = 115700 (0.167 sec)
INFO:tensorflow:global_step/sec: 595.847
INFO:tensorflow:loss = 41030190000.0, step = 115800 (0.169 sec)
INFO:tensorflow:global_step/sec: 592.32
INFO:tensorflow:loss = 142807430000.0, step = 115900 (0.168 sec)
INFO:tensorflow:global_step/sec: 592.324
INFO:tensorflow:loss = 68154090000.0, step = 116000 (0.169 sec)
INFO:tensorflow:global_step/sec: 595.847
INFO:tensor

INFO:tensorflow:global_step/sec: 592.32
INFO:tensorflow:loss = 72903480000.0, step = 123000 (0.169 sec)
INFO:tensorflow:global_step/sec: 588.84
INFO:tensorflow:loss = 45007004000.0, step = 123100 (0.170 sec)
INFO:tensorflow:global_step/sec: 595.847
INFO:tensorflow:loss = 68214346000.0, step = 123200 (0.168 sec)
INFO:tensorflow:global_step/sec: 588.836
INFO:tensorflow:loss = 57030530000.0, step = 123300 (0.170 sec)
INFO:tensorflow:global_step/sec: 592.323
INFO:tensorflow:loss = 147854590000.0, step = 123400 (0.169 sec)
INFO:tensorflow:global_step/sec: 592.321
INFO:tensorflow:loss = 137266450000.0, step = 123500 (0.169 sec)
INFO:tensorflow:global_step/sec: 592.324
INFO:tensorflow:loss = 58409837000.0, step = 123600 (0.170 sec)
INFO:tensorflow:global_step/sec: 592.321
INFO:tensorflow:loss = 39716766000.0, step = 123700 (0.168 sec)
INFO:tensorflow:global_step/sec: 588.834
INFO:tensorflow:loss = 59553853000.0, step = 123800 (0.170 sec)
INFO:tensorflow:global_step/sec: 599.418
INFO:tensorflo

INFO:tensorflow:global_step/sec: 588.837
INFO:tensorflow:loss = 86082490000.0, step = 130800 (0.170 sec)
INFO:tensorflow:global_step/sec: 595.848
INFO:tensorflow:loss = 63136220000.0, step = 130900 (0.168 sec)
INFO:tensorflow:global_step/sec: 599.417
INFO:tensorflow:loss = 58991026000.0, step = 131000 (0.167 sec)
INFO:tensorflow:global_step/sec: 585.394
INFO:tensorflow:loss = 47525683000.0, step = 131100 (0.171 sec)
INFO:tensorflow:global_step/sec: 599.416
INFO:tensorflow:loss = 37488660000.0, step = 131200 (0.167 sec)
INFO:tensorflow:global_step/sec: 588.836
INFO:tensorflow:loss = 79301845000.0, step = 131300 (0.170 sec)
INFO:tensorflow:global_step/sec: 588.838
INFO:tensorflow:loss = 69875830000.0, step = 131400 (0.170 sec)
INFO:tensorflow:global_step/sec: 592.321
INFO:tensorflow:loss = 137936860000.0, step = 131500 (0.169 sec)
INFO:tensorflow:global_step/sec: 592.322
INFO:tensorflow:loss = 34707980000.0, step = 131600 (0.170 sec)
INFO:tensorflow:global_step/sec: 588.838
INFO:tensorfl

...

INFO:tensorflow:global_step/sec: 581.985
INFO:tensorflow:loss = 90064320000.0, step = 162000 (0.172 sec)
INFO:tensorflow:global_step/sec: 578.632
INFO:tensorflow:loss = 129980480000.0, step = 162100 (0.173 sec)
INFO:tensorflow:global_step/sec: 595.847
INFO:tensorflow:loss = 51727670000.0, step = 162200 (0.168 sec)
INFO:tensorflow:global_step/sec: 592.324
INFO:tensorflow:loss = 264781140000.0, step = 162300 (0.169 sec)
INFO:tensorflow:global_step/sec: 592.321
INFO:tensorflow:loss = 75187530000.0, step = 162400 (0.169 sec)
INFO:tensorflow:global_step/sec: 595.848
INFO:tensorflow:loss = 85412270000.0, step = 162500 (0.168 sec)
INFO:tensorflow:global_step/sec: 592.32
INFO:tensorflow:loss = 130182120000.0, step = 162600 (0.169 sec)
INFO:tensorflow:global_step/sec: 532.461
INFO:tensorflow:loss = 89003850000.0, step = 162700 (0.188 sec)
INFO:tensorflow:global_step/sec: 578.624
INFO:tensorflow:loss = 54164570000.0, step = 162800 (0.173 sec)
INFO:tensorflow:global_step/sec: 585.394
INFO:tensorf

INFO:tensorflow:global_step/sec: 592.322
INFO:tensorflow:loss = 76682535000.0, step = 169800 (0.169 sec)
INFO:tensorflow:global_step/sec: 592.321
INFO:tensorflow:loss = 49058808000.0, step = 169900 (0.169 sec)
INFO:tensorflow:global_step/sec: 595.849
INFO:tensorflow:loss = 182171680000.0, step = 170000 (0.168 sec)
INFO:tensorflow:global_step/sec: 592.321
INFO:tensorflow:loss = 135969880000.0, step = 170100 (0.169 sec)
INFO:tensorflow:global_step/sec: 581.991
INFO:tensorflow:loss = 70414730000.0, step = 170200 (0.172 sec)
INFO:tensorflow:global_step/sec: 592.322
INFO:tensorflow:loss = 121948410000.0, step = 170300 (0.169 sec)
INFO:tensorflow:global_step/sec: 588.838
INFO:tensorflow:loss = 52944536000.0, step = 170400 (0.170 sec)
INFO:tensorflow:global_step/sec: 588.838
INFO:tensorflow:loss = 76257810000.0, step = 170500 (0.170 sec)
INFO:tensorflow:global_step/sec: 595.847
INFO:tensorflow:loss = 124765274000.0, step = 170600 (0.168 sec)
INFO:tensorflow:global_step/sec: 595.849
INFO:tenso

INFO:tensorflow:global_step/sec: 588.839
INFO:tensorflow:loss = 63729574000.0, step = 177600 (0.170 sec)
INFO:tensorflow:global_step/sec: 541.094
INFO:tensorflow:loss = 205665940000.0, step = 177700 (0.185 sec)
INFO:tensorflow:global_step/sec: 588.837
INFO:tensorflow:loss = 67951063000.0, step = 177800 (0.170 sec)
INFO:tensorflow:global_step/sec: 592.321
INFO:tensorflow:loss = 153990170000.0, step = 177900 (0.169 sec)
INFO:tensorflow:global_step/sec: 592.323
INFO:tensorflow:loss = 82547740000.0, step = 178000 (0.169 sec)
INFO:tensorflow:global_step/sec: 588.837
INFO:tensorflow:loss = 121864700000.0, step = 178100 (0.170 sec)
INFO:tensorflow:global_step/sec: 553.052
INFO:tensorflow:loss = 107876590000.0, step = 178200 (0.181 sec)
INFO:tensorflow:global_step/sec: 572.013
INFO:tensorflow:loss = 87645420000.0, step = 178300 (0.176 sec)
INFO:tensorflow:global_step/sec: 562.111
INFO:tensorflow:loss = 177356130000.0, step = 178400 (0.177 sec)
INFO:tensorflow:global_step/sec: 572.286
INFO:tens

INFO:tensorflow:global_step/sec: 588.837
INFO:tensorflow:loss = 79992770000.0, step = 185400 (0.169 sec)
INFO:tensorflow:global_step/sec: 592.322
INFO:tensorflow:loss = 37657800000.0, step = 185500 (0.169 sec)
INFO:tensorflow:global_step/sec: 592.322
INFO:tensorflow:loss = 116938695000.0, step = 185600 (0.169 sec)
INFO:tensorflow:global_step/sec: 588.836
INFO:tensorflow:loss = 70617720000.0, step = 185700 (0.170 sec)
INFO:tensorflow:global_step/sec: 541.095
INFO:tensorflow:loss = 98249440000.0, step = 185800 (0.185 sec)
INFO:tensorflow:global_step/sec: 588.838
INFO:tensorflow:loss = 223165350000.0, step = 185900 (0.170 sec)
INFO:tensorflow:global_step/sec: 595.848
INFO:tensorflow:loss = 137457470000.0, step = 186000 (0.168 sec)
INFO:tensorflow:global_step/sec: 585.394
INFO:tensorflow:loss = 171004460000.0, step = 186100 (0.171 sec)
INFO:tensorflow:global_step/sec: 592.323
INFO:tensorflow:loss = 154438120000.0, step = 186200 (0.169 sec)
INFO:tensorflow:global_step/sec: 592.321
INFO:tens

INFO:tensorflow:global_step/sec: 592.322
INFO:tensorflow:loss = 74175340000.0, step = 193200 (0.169 sec)
INFO:tensorflow:global_step/sec: 592.321
INFO:tensorflow:loss = 293130340000.0, step = 193300 (0.169 sec)
INFO:tensorflow:global_step/sec: 581.992
INFO:tensorflow:loss = 71134680000.0, step = 193400 (0.172 sec)
INFO:tensorflow:global_step/sec: 603.025
INFO:tensorflow:loss = 131200370000.0, step = 193500 (0.167 sec)
INFO:tensorflow:global_step/sec: 595.85
INFO:tensorflow:loss = 100751150000.0, step = 193600 (0.167 sec)
INFO:tensorflow:global_step/sec: 592.32
INFO:tensorflow:loss = 148525600000.0, step = 193700 (0.169 sec)
INFO:tensorflow:global_step/sec: 595.85
INFO:tensorflow:loss = 118741090000.0, step = 193800 (0.168 sec)
INFO:tensorflow:global_step/sec: 592.323
INFO:tensorflow:loss = 28396743000.0, step = 193900 (0.169 sec)
INFO:tensorflow:global_step/sec: 585.392
INFO:tensorflow:loss = 64115490000.0, step = 194000 (0.171 sec)
INFO:tensorflow:global_step/sec: 592.323
INFO:tensorf

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [25]:
input_predict_func = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=16, 
                                                         num_epochs=1, shuffle=False)

In [26]:
predictions_generator = model.predict(input_fn=input_predict_func)

In [27]:
predictions = list(predictions_generator)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Juancho\AppData\Local\Temp\tmplfhg0puc\model.ckpt-200000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [28]:
predictions

[{'predictions': array([251600.77], dtype=float32)},
 {'predictions': array([156365.53], dtype=float32)},
 {'predictions': array([196693.86], dtype=float32)},
 {'predictions': array([375003.25], dtype=float32)},
 {'predictions': array([219860.89], dtype=float32)},
 {'predictions': array([160676.11], dtype=float32)},
 {'predictions': array([166652.98], dtype=float32)},
 {'predictions': array([63551.473], dtype=float32)},
 {'predictions': array([217803.66], dtype=float32)},
 {'predictions': array([441410.03], dtype=float32)},
 {'predictions': array([270673.06], dtype=float32)},
 {'predictions': array([152811.89], dtype=float32)},
 {'predictions': array([149523.1], dtype=float32)},
 {'predictions': array([199131.7], dtype=float32)},
 {'predictions': array([226255.64], dtype=float32)},
 {'predictions': array([127840.8], dtype=float32)},
 {'predictions': array([84270.41], dtype=float32)},
 {'predictions': array([171234.81], dtype=float32)},
 {'predictions': array([47863.91], dtype=float32)}

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [29]:
final_predictions = []

for pred in predictions:
    final_predictions.append(pred["predictions"])

In [30]:
from sklearn.metrics import mean_squared_error

In [31]:
mean_squared_error(y_test, final_predictions)**0.5

75758.28714331587

# Great Job!